# Measuring correlation between ideological distance and comment sentiments

This notebook uses Micheles methods.

Ideological distance calculation uses the node title embeddings.

Comment sentiments are from the edge values.

Output ranges from -1 to 1, showing negative and positive correlation respectively.

In [ ]:
import sys
sys.path.append('../')
from py_files.michele_algo import *

In [ ]:
# networks to run and print results from
network_list = [
    ['gaming', '../data/date_folders/april_4/graphs/'],
    #['tarkov_network', '../data/date_folders/march_19/graphs/'],
    #['CallOfDuty', '../data/date_folders/april_18/graphs/'],
    #['FIFA', '../data/date_folders/april_18/graphs/'],
    ['politics_nan', '../data/date_folders/march_23/graphs/'],
    ['worldnews', '../data/date_folders/april_23/graphs/'],
    ['PoliticalDiscussion', '../data/date_folders/april_23/graphs/'],
    ['ukpolitics', '../data/date_folders/april_23/graphs/'],
    ['democrats', '../data/date_folders/april_23/graphs/'],
    ['Republican', '../data/date_folders/april_23/graphs/']
]

In [ ]:
for subreddit, folderpath in network_list:
    print(subreddit)
    print(driver(subreddit, folderpath))